In [24]:
import rdflib

# Specify the path to your Turtle file
turtle_file = 'output.ttl'

# Create a new RDFLib Graph
g = rdflib.Graph()

# Attempt to load the Turtle file into the graph
try:
    g.parse(turtle_file, format="turtle")
    print("Turtle file loaded successfully.")
except Exception as e:
    print(f"Error loading Turtle file: {e}")
        
# Print the number of triples in the graph
print(f"Number of triples in the graph: {len(g)}")

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#time, Converter=<function parse_time at 0x000001FD797A31A0>
Traceback (most recent call last):
  File "d:\Anaconda\envs\myenv\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
  File "d:\Anaconda\envs\myenv\Lib\site-packages\isodate\isotime.py", line 148, in parse_time
    raise ISO8601Error('Unrecognised ISO 8601 time format: %r' % timestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 time format: '2x90 min/week'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#time, Converter=<function parse_time at 0x000001FD797A31A0>
Traceback (most recent call last):
  File "d:\Anaconda\envs\myenv\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
 

Turtle file loaded successfully.
Number of triples in the graph: 8166


# Template 1: one given tail entity and one given relation - search head entity

In [25]:
def search_head_entities(tail_entity_uri, relation_uri):
    # Prepare the SPARQL query with dynamic relation and tail entity URIs
    query = f"""
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX default1: <https://nfdi.fiz-karlsruhe.de/ontology/>
    PREFIX emmo: <http://emmo.info/emmo#>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX modsci: <https://w3id.org/skgo/modsci#>
    PREFIX mwo: <http://purls.helmholtz-metadaten.de/mwo/>
    PREFIX nfdicore: <http://nfdi.fiz-karlsruhe.de/ontology/>
    PREFIX ns1: <https://w3id.org/scholarlydata/ontology/conference-ontology.owl#>
    PREFIX ns2: <http://purl.obolibrary.org/obo/>
    PREFIX ns3: <http://www.ebi.ac.uk/swo/>
    PREFIX ns4: <http://www.geneontology.org/formats/oboInOwl#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX schema: <https://schema.org/>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX void: <http://rdfs.org/ns/void#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    SELECT ?headEntity
    WHERE {{
      ?headEntity {relation_uri} <{tail_entity_uri}> .
    }}
    """

    # Execute the query and collect results as tuples
    triples = []
    try:
        results = g.query(query)
        for row in results:
            head_entity = str(row.headEntity)
            if head_entity.startswith("http"):
                head_entity = head_entity  # URIs are output directly without <>
            else:
                head_entity = f":{head_entity}"  # Prefixed names are prefixed with a colon

            # Assuming relation_uri is a prefixed name, not a full URI, so it's output directly
            tail_entity = tail_entity_uri  # URIs are output directly without <>

            triple = f"('{head_entity}', '{relation_uri}', '{tail_entity}')"
            triples.append(triple)
    except Exception as e:
        print(f"Error executing SPARQL query: {e}")
        return []

    return triples

# Example usage
relation_uri = 'mwo:hasWebsite'
tail_entity_uri = 'http://demo.fiz-karlsruhe.de/matwerk/E340077'
triples = search_head_entities(tail_entity_uri,relation_uri)
for triple in triples:
    print(triple)

('http://demo.fiz-karlsruhe.de/matwerk/E1012754', 'mwo:hasWebsite', 'http://demo.fiz-karlsruhe.de/matwerk/E340077')


# Template 2: one given head entity and one given relation - search tail entity, include literal

In [26]:
import rdflib

def search_tail_entities(head_entity_uri, relation_uri):
    # Assuming 'g' is your RDFLib Graph object loaded with your Turtle data
    global g

    # Prepare the SPARQL query with dynamic head entity and relation URIs
    query = f"""
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX default1: <https://nfdi.fiz-karlsruhe.de/ontology/>
    PREFIX emmo: <http://emmo.info/emmo#>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX modsci: <https://w3id.org/skgo/modsci#>
    PREFIX mwo: <http://purls.helmholtz-metadaten.de/mwo/>
    PREFIX nfdicore: <http://nfdi.fiz-karlsruhe.de/ontology/>
    PREFIX ns1: <https://w3id.org/scholarlydata/ontology/conference-ontology.owl#>
    PREFIX ns2: <http://purl.obolibrary.org/obo/>
    PREFIX ns3: <http://www.ebi.ac.uk/swo/>
    PREFIX ns4: <http://www.geneontology.org/formats/oboInOwl#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX schema: <https://schema.org/>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX void: <http://rdfs.org/ns/void#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    SELECT ?tailEntity
    WHERE {{
      <{head_entity_uri}> {relation_uri} ?tailEntity .
    }}
    """

    # Execute the query and collect results as tuples
    triples = []
    try:
        results = g.query(query)
        for row in results:
            # Construct a tuple for each triple
            tail_entity = row.tailEntity.toPython() if isinstance(row.tailEntity, rdflib.URIRef) else f"\"{row.tailEntity}\""
            triple = (head_entity_uri, relation_uri, tail_entity)
            triples.append(triple)
    except Exception as e:
        print(f"Error executing SPARQL query: {e}")
        return []

    return triples

# Example usage
head_entity_uri = 'http://demo.fiz-karlsruhe.de/matwerk/E1004237'
relation_uri = 'mwo:description'

turtle_triples = search_tail_entities(head_entity_uri, relation_uri)
for triple in turtle_triples:
    print(triple)

('http://demo.fiz-karlsruhe.de/matwerk/E1004237', 'mwo:description', '"GitHub, Inc., is an Internet hosting service for software development and version control using Git. It provides the distributed version control of Git plus access control, bug tracking, software feature requests, task management, continuous integration, and wikis for every project."')


# Template 3:  one given head entity and one given relation, firstly find one tail entity, then find all head entities with same tail entity and same relation

In [27]:
import rdflib

def search_related_head_entities(initial_head_entity_uri, relation_uri):
    # Define all the prefix namespaces
    mwo_prefix = "http://purls.helmholtz-metadaten.de/mwo/"
    dc_prefix = "http://purl.org/dc/elements/1.1/"
    dcterms_prefix = "http://purl.org/dc/terms/"
    default1_prefix = "https://nfdi.fiz-karlsruhe.de/ontology/"
    emmo_prefix = "http://emmo.info/emmo#"
    foaf_prefix = "http://xmlns.com/foaf/0.1/"
    modsci_prefix = "https://w3id.org/skgo/modsci#"
    nfdicore_prefix = "http://nfdi.fiz-karlsruhe.de/ontology/"
    ns1_prefix = "https://w3id.org/scholarlydata/ontology/conference-ontology.owl#"
    ns2_prefix = "http://purl.obolibrary.org/obo/"
    ns3_prefix = "http://www.ebi.ac.uk/swo/"
    ns4_prefix = "http://www.geneontology.org/formats/oboInOwl#"
    owl_prefix = "http://www.w3.org/2002/07/owl#"
    rdf_prefix = "http://www.w3.org/1999/02/22-rdf-syntax-ns#"
    rdfs_prefix = "http://www.w3.org/2000/01/rdf-schema#"
    schema_prefix = "https://schema.org/"
    skos_prefix = "http://www.w3.org/2004/02/skos/core#"
    void_prefix = "http://rdfs.org/ns/void#"
    xsd_prefix = "http://www.w3.org/2001/XMLSchema#"

    # Combine the two queries into one
    combined_query = f"""
    PREFIX mwo: <{mwo_prefix}>
    PREFIX dc: <{dc_prefix}>
    PREFIX dcterms: <{dcterms_prefix}>
    PREFIX default1: <{default1_prefix}>
    PREFIX emmo: <{emmo_prefix}>
    PREFIX foaf: <{foaf_prefix}>
    PREFIX modsci: <{modsci_prefix}>
    PREFIX nfdicore: <{nfdicore_prefix}>
    PREFIX ns1: <{ns1_prefix}>
    PREFIX ns2: <{ns2_prefix}>
    PREFIX ns3: <{ns3_prefix}>
    PREFIX ns4: <{ns4_prefix}>
    PREFIX owl: <{owl_prefix}>
    PREFIX rdf: <{rdf_prefix}>
    PREFIX rdfs: <{rdfs_prefix}>
    PREFIX schema: <{schema_prefix}>
    PREFIX skos: <{skos_prefix}>
    PREFIX void: <{void_prefix}>
    PREFIX xsd: <{xsd_prefix}>
    
    SELECT ?otherHeadEntity ?tailEntity
    WHERE {{
        {{
            SELECT ?tailEntity
            WHERE {{
                <{initial_head_entity_uri}> {relation_uri} ?tailEntity .
            }}
        }}
        ?otherHeadEntity {relation_uri} ?tailEntity .
        FILTER (?otherHeadEntity != <{initial_head_entity_uri}>)  # Exclude the initial head entity
    }}
    """

    try:
        results = g.query(combined_query)
        triples = [(str(row.otherHeadEntity), relation_uri, str(row.tailEntity)) for row in results]
    except Exception as e:
        print(f"Error executing the SPARQL query: {e}")
        return []

    return triples

# Example usage
initial_head_entity_uri = 'http://demo.fiz-karlsruhe.de/matwerk/E15879'
relation_uri = 'mwo:hasAffiliation'

related_head_entities = search_related_head_entities(initial_head_entity_uri, relation_uri)
for entity in related_head_entities:
    print(entity)

('http://demo.fiz-karlsruhe.de/matwerk/E1245566', 'mwo:hasAffiliation', 'http://demo.fiz-karlsruhe.de/matwerk/E1016')
('http://demo.fiz-karlsruhe.de/matwerk/E16052', 'mwo:hasAffiliation', 'http://demo.fiz-karlsruhe.de/matwerk/E1016')
('http://demo.fiz-karlsruhe.de/matwerk/E9779', 'mwo:hasAffiliation', 'http://demo.fiz-karlsruhe.de/matwerk/E1016')
('http://demo.fiz-karlsruhe.de/matwerk/E14862', 'mwo:hasAffiliation', 'http://demo.fiz-karlsruhe.de/matwerk/E1016')


# Template 4:  one given tail entity and one given relation, firstly find one head entity, then find all tail entities with same tail entity and same relation

# Search for all relevant triples 

In [28]:
def search_all_possible_triples(entity_uri, relation_uri):
    # Call the custom functions to find relevant triples
    triples_query1 = search_head_entities(entity_uri, relation_uri)
    triples_query2 = search_tail_entities(entity_uri, relation_uri)
    triples_query3 = search_related_head_entities(entity_uri, relation_uri)
    
    # Combine the results from all three queries
    all_triples = triples_query1 + triples_query2 + triples_query3
def search_all_possible_triples(entity_uri, relation_uri):
    if relation_uri == 'rdf:type':
        # Call the custom functions to find relevant triples for rdf:type
        triples_query1 = search_head_entities(entity_uri, relation_uri)
        triples_query2 = search_tail_entities(entity_uri, relation_uri)
        
        # Combine the results from the two queries
        all_triples = triples_query1 + triples_query2
    else:
        # Call the custom functions to find relevant triples
        triples_query1 = search_head_entities(entity_uri, relation_uri)
        triples_query2 = search_tail_entities(entity_uri, relation_uri)
        triples_query3 = search_related_head_entities(entity_uri, relation_uri)
        
        # Combine the results from all three queries
        all_triples = triples_query1 + triples_query2 + triples_query3

    # Remove duplicates by converting the list of triples to a set and back to a list
    all_triples = list(set(all_triples))
    
    return all_triples

    # Remove duplicates by converting the list of triples to a set and back to a list
    all_triples = list(set(all_triples))
    
    return all_triples

# Read the file from Dataprocessing

In [29]:
import pandas as pd
df = pd.read_excel('relevant_entities_relations.xlsx')
df.head()

,Text,Named Entities,Predicate Verbs,Similar Relations,Relation URIs,Relation_uris_with_Namespace,Similar Entities,Entity URIs
0,Who is working in the Computational Materials ...,the Computational Materials Science,work,"['has work package', 'has expertise in', 'has ...",['https://nfdi.fiz-karlsruhe.de/ontology/examp...,"['mwo:hasWorkPackage', 'mwo:hasExpertiseIn', '...","['Computational Material Science', 'computatio...",['http://demo.fiz-karlsruhe.de/matwerk/E67431'...
1,What are the research projects associated to E...,EMMO,research project associate,"['has related Project', 'related participant p...","['https://schema.org/dateCreated', 'http://nfd...","['nfdicore:relatedProject', 'mwo:relatedPartic...","['ruby', 'R. S. Elliott and E. B. Tadmor, ""Kno...",['http://demo.fiz-karlsruhe.de/matwerk/E837572...
2,"Who are the contributors of the data ""datasets""?",datasets,contributor,"['has contributor', 'related participant proje...",['http://www.geneontology.org/formats/oboInOwl...,"['mwo:hasContributor', 'mwo:relatedParticipant...","['Image data', 'Framework for curation and dis...",['http://demo.fiz-karlsruhe.de/matwerk/E119683...
3,"Who is working with Researcher ""Ebrahim Norouz...",Ebrahim Norouzi,work,"['has work package', 'has expertise in', 'has ...",['https://nfdi.fiz-karlsruhe.de/ontology/examp...,"['mwo:hasWorkPackage', 'mwo:hasExpertiseIn', '...","['Dr. Amir Laadhar', 'Mirza Mohtashim Alam', '...",['http://demo.fiz-karlsruhe.de/matwerk/E10181'...
4,"Who is the email address of ""ParaView""?",ParaView,email address,"['has email address ', 'has postal address', '...","['http://purl.obolibrary.org/obo/IAO_0000119',...","['mwo:emailAddress', 'mwo:hasPostalAddress', '...","['paraview', 'data portal', 'Standardised docu...",['http://demo.fiz-karlsruhe.de/matwerk/E123109...


In [30]:
import ast
# Initialize a new column in the DataFrame to store the found triples for each row
df['Found Triples'] = None

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    found_triples = []  # Initialize an empty list to store the found triples for the current row

    # Parse the 'Entity URIs' and 'relationship_uris_withNS' columns from the current row
    entities_uris = ast.literal_eval(row['Entity URIs'])
    relationship_uris = ast.literal_eval(row['Relation_uris_with_Namespace'])

    # Iterate through each combination of entity URI and relationship URI
    for entity_uri in entities_uris:
        for relation_uri in relationship_uris:
            triples = search_all_possible_triples(entity_uri, relation_uri)
            for triple in triples:
                if triple not in found_triples:  # Check if the triple is not already in the list
                    found_triples.append(triple)
    
    # Store the found triples for the current row in the new column
    df.at[index, 'Found Triples'] =str(list(found_triples))  
df.to_excel('sparql_output.xlsx', index=False)  # Save the updated DataFrame to a new Excel file
df.head()

,Text,Named Entities,Predicate Verbs,Similar Relations,Relation URIs,Relation_uris_with_Namespace,Similar Entities,Entity URIs,Found Triples
0,Who is working in the Computational Materials ...,the Computational Materials Science,work,"['has work package', 'has expertise in', 'has ...",['https://nfdi.fiz-karlsruhe.de/ontology/examp...,"['mwo:hasWorkPackage', 'mwo:hasExpertiseIn', '...","['Computational Material Science', 'computatio...",['http://demo.fiz-karlsruhe.de/matwerk/E67431'...,"[""('http://demo.fiz-karlsruhe.de/matwerk/E1031..."
1,What are the research projects associated to E...,EMMO,research project associate,"['has related Project', 'related participant p...","['https://schema.org/dateCreated', 'http://nfd...","['nfdicore:relatedProject', 'mwo:relatedPartic...","['ruby', 'R. S. Elliott and E. B. Tadmor, ""Kno...",['http://demo.fiz-karlsruhe.de/matwerk/E837572...,[('http://demo.fiz-karlsruhe.de/matwerk/E83757...
2,"Who are the contributors of the data ""datasets""?",datasets,contributor,"['has contributor', 'related participant proje...",['http://www.geneontology.org/formats/oboInOwl...,"['mwo:hasContributor', 'mwo:relatedParticipant...","['Image data', 'Framework for curation and dis...",['http://demo.fiz-karlsruhe.de/matwerk/E119683...,[('http://demo.fiz-karlsruhe.de/matwerk/E11968...
3,"Who is working with Researcher ""Ebrahim Norouz...",Ebrahim Norouzi,work,"['has work package', 'has expertise in', 'has ...",['https://nfdi.fiz-karlsruhe.de/ontology/examp...,"['mwo:hasWorkPackage', 'mwo:hasExpertiseIn', '...","['Dr. Amir Laadhar', 'Mirza Mohtashim Alam', '...",['http://demo.fiz-karlsruhe.de/matwerk/E10181'...,[('http://demo.fiz-karlsruhe.de/matwerk/E29921...
4,"Who is the email address of ""ParaView""?",ParaView,email address,"['has email address ', 'has postal address', '...","['http://purl.obolibrary.org/obo/IAO_0000119',...","['mwo:emailAddress', 'mwo:hasPostalAddress', '...","['paraview', 'data portal', 'Standardised docu...",['http://demo.fiz-karlsruhe.de/matwerk/E123109...,[('http://demo.fiz-karlsruhe.de/matwerk/E12310...


In [31]:
# Select two coloumns"Text""Found Triples" as new dataframe
df_nwe= df[['Text', 'Found Triples']]
df_nwe.to_excel('for_verbalisation.xlsx', index=False)  # Save the updated DataFrame to a new Excel file